# Download OSM Data from Web #

This method uses de OSM Overpass API to get OSM data from the Web

In [ ]:
from gasp3.adv.osm import download_by_boundary

inBoundary = r'F:\rs_dez\lmt\lmt_ua12_ext.shp'
outOsm = r'F:\rs_dez\osmdata\osm_coimbra.xml'

EPSG = None

download_by_boundary(inBoundary, outOsm, EPSG, GetUrl=None)

# Convert OSM data into Feature Class #

In [1]:
from gasp3.gt.to.shp import osm_to_featcls

osm_to_featcls(
    r'F:\rs_dez\osmdata\osm_coimbra.xml',
    r'F:\rs_dez\osmdata', useXmlName=True, outepsg=32629)

'F:\\rs_dez\\osmdata'